In [234]:
import numpy as np
from math import exp, log
from numpy.random import randint
import random

In [235]:
def quasi_cyclic() :
    I = np.eye(27)
    O = np.zeros((27, 27))
    Is = [np.roll(I, -p, 0) for p in range(27)]
    H = np.block([[Is[0], O, O, O, Is[0], Is[0], O, O, Is[0], O, O, Is[0], Is[1], Is[0], O, O, O, O, O, O, O, O, O, O], 
                 [Is[22], Is[0], O, O, Is[17], O, Is[0], Is[0], Is[12], O, O, O, O, Is[0], Is[0], O, O, O, O, O, O, O, O, O],
                 [Is[6], O, Is[0], O, Is[10], O, O, O, Is[24], O, Is[0], O, O, O, Is[0], Is[0], O, O, O, O, O, O, O, O],
                 [Is[2], O, O, Is[0], Is[20], O, O ,O, Is[25], Is[0], O, O, O, O, O, Is[0], Is[0], O, O, O, O, O, O, O],
                 [Is[23], O, O, O, Is[3], O, O, O, Is[0], O, Is[7], Is[11], O, O, O, O, Is[0], Is[0], O, O, O, O, O, O],
                 [Is[24], O, Is[23], Is[1], Is[17], O, Is[3], O, Is[10], O, O, O, O, O, O, O, O, Is[0], Is[0], O, O, O, O, O],
                 [Is[25], O, O, O, Is[8], O, O, O, Is[7], Is[18], O, O, O, O, O, O, O, O, Is[0], Is[0], O, O, O, O],
                 [Is[13], Is[24], O, Is[16], Is[0], O, Is[8], O, Is[6], O, O, O, O, O, O, O, O, O, O, Is[0], Is[0], O, O, O],
                 [Is[7], Is[20], O, O, Is[22], Is[10], O, O, Is[23], O, O, O, O, O, O, O, O, O, O, O, Is[0], Is[0], O, O],
                 [Is[11], O, O, O, Is[19], O, O, O, Is[13], O, Is[3], Is[17], O, O, O, O, O, O, O, O, O, Is[0], Is[0], O],
                 [Is[25], O, Is[8], O, Is[23], Is[18], O, Is[14], Is[9], O, O, O, O, O, O, O, O, O, O, O, O, O, Is[0], Is[0]],
                 [Is[3], O, O, O, Is[16], O, O, Is[2], Is[25], Is[5], O, O, O, O, O, O, O, O, O, O, O, O, O, Is[0]]])
    return H

In [236]:
def build_regular_matrix(m, n) :
    #cols = randint(3, 8, n)
    #rows = randint(4, 11, m)
    col = 3
    row = col * (n // m) 
    cols_cnt = [0 for _ in range(n)]
    H = np.zeros((m, n), dtype = int)
    
    for i in range(m) :
        #indices = randint(0, n, rows[i])
        indices = random.sample(range(n), row)
        for index in indices :
            j = index
            while (cols_cnt[j] > col - 1) :
                j = (j + 1) % n
            cols_cnt[j] += 1
            H[i, j] = 1
    return H

In [237]:
def parity_matrix(n, p) :
    if n == 648 :
        m = 324
        H = quasi_cyclic()
    else :
        if p <= 0.02 :
            m = n // 5
        elif p <= 0.04 :
            m = n // 3
        elif p <= 0.08 :
            m = n // 2
        else :
            m = 2 * n // 3
        H = build_regular_matrix(m, n)
    return H, m


In [238]:
def lookup_tables(H, m, n, k) :
    LL_index = []
    LL_groups = 0
    for row in np.transpose(H) :
        LL_index.append(LL_groups)
        LL_groups += sum(row)
    LL_index.append(k)
    
    cs_index = []
    cs_groups = 0
    for row in H :
        cs_index.append(cs_groups)
        cs_groups += sum(row)
    cs_index.append(k)
    
    cs_list = []
    bit_cs = [0 for _ in range(n)]
    for row in range(m) :
        for bit in range(n) :
            if H[row, bit] == 1 :
                cs_list.append([bit, LL_index[bit] + bit_cs[bit]])
                bit_cs[bit] += 1
    cs_list.append([0,0])           
    return np.array(LL_index), np.array(cs_index), np.array(cs_list)

In [239]:
def non_zeros(M) :
    return sum([sum(row) for row in M])

In [240]:
def syndrome(M1, x) :
    return [i % 2 for i in np.matmul(M1, np.transpose(np.array(x)))]

In [241]:
def produce_a2f(p, Na, Nf, Ma, Mf) :
    a2f = []
    for i in range(Ma) :
        a = exp(-i/Na)
        f = (1 + a)/(1 - a)
        z = log(f)
        j = int(Nf * z + 0.5) 
        if j <= 0 :
            j = 1
        if j > Mf :
            j = Mf
        a2f.append(j)
    return a2f

In [242]:
def produce_f2a(p, Na, Nf, Ma, Mf) :
    f2a = [0]
    for i in range(1, Mf + 1) :
        f = exp(i/Nf)
        a = (f - 1)/(f + 1)
        z = -log(a)
        j = int(Na * z + 0.5) 
        if j <= 0 :
            j = 1
        if j > Ma :
            j = Ma
        f2a.append(j)
    return f2a

In [243]:
def LL_init1(Na, Nf, k, p) :
    LL_reg = [0]
    f_init = Nf * log((1-p)/p)
    a_init = Na * log(1 - 2 * p)
    for i in range(1, k + 1) :
        LL_reg.append(a_init)
    return f_init, LL_reg

In [244]:
def LL_init(k, p) :
    LL_reg = []
    f_init = (1-p)/p
    a_init = (1 - 2 * p)
    for i in range(k) :
        LL_reg.append(a_init)
    LL_reg.append(0)
    return f_init, LL_reg

In [245]:
def cs_msgs_2_bits1(m, d, cs_index, cs_list, LL_reg, Ma) : 
    for i in range(m) :
        sign = d[i]
        big_alpha = 0
        j1 = cs_index[i]
        j2 = cs_index[i + 1]
        for j in range(j1, j2) :
            a1 = LL_reg[cs_list[j][1]]
            if a1 < 0 :
                sign = 1 - sign
                big_alpha -= a1
        for j in range(j1, j2) :
            a1 = LL_reg[cs_list[j][1]]
            p_sign = sign
            if a1 < 0 :
                p_alpha = big_alpha + a1
            else :
                p_alpha = big_alpha + a1
            if p_alpha <= 0 :
                p_alpha = 1
            if p_alpha > Ma :
                p_alpha = Ma
            if p_sign == 0 :
                LL_reg[cs_list[j][1]] = p_alpha
            else :
                LL_reg[cs_list[j][1]] = - p_alpha
    return LL_reg

In [246]:
def cs_msgs_2_bits(m, d, cs_index, cs_list, LL_reg) : 
    for i in range(m) :
        alpha = (-1) ** (d[i])
        j1 = cs_index[i]
        j2 = cs_index[i + 1]
        for j in range(j1, j2) :
            a1 = LL_reg[cs_list[j][1]]
            alpha *= a1 
        if 0 > alpha > -0.0001 :
            alpha = -0.0001
        if 0.0001 > alpha >= 0 :
            alpha = 0.0001
        for j in range(j1, j2) :
            a1 = LL_reg[cs_list[j][1]]
            f = (a1 + alpha) / (a1 - alpha)
            if f < 0.001 :
                f = 0.001
            if 0.999 < f < 1.0 :
                f = 0.999
            if 1.0 <= f < 1.0001 :
                f = 1.001
            if f > 1000 :
                f = 1000
            LL_reg[cs_list[j][1]] = f
    return LL_reg

In [247]:
def bit_msgs_2_cs1(n, f_init, a2f, Mf, LL_reg, y) :
    y1 = []
    for i in range(1, n + 1) :
        j1, j2 = LL_index[i], LL_index[i + 1]
        f_tot = f_init
        for j in range(j1, j2) :
            u = LL_reg[j]
            if u > 0 :
                u = a2f[u]
            else :
                u = - a2f[-u]
            LL_reg[j] = u
            f_tot += u
        
        for j in range(j1, j2) :
            x = f_tot - LL_reg[j]
            if x < 0 :
                p_sign = 1
                x = -x
            else :
                p_sign = 0
            if x < 1 :
                x = 1
            if x > Mf :
                x = Mf
            if p_sign == 1 :
                LL_reg[j] = - f2a[x]
            else :
                LL_reg[j] = f2a[x]
    
        if f_tot < 0 :
            y1.append(1 - y[i])
        else : 
            y1.append(y[i])
        return y1, LL_reg

In [248]:
def bit_msgs_2_cs(n, LL_reg, LL_index, f_init, y) :
    y1 = []
    for i in range(n) :
        j1, j2 = LL_index[i], LL_index[i + 1]
        f_tot = f_init
        for j in range(j1, j2) :
            f = LL_reg[j]
            f_tot *= f
            
        for j in range(j1, j2) :
            f = f_tot - LL_reg[j]
            a = (f_tot - f) / (f_tot + f)
            if a < 0 :
                if a > -0.001 :
                    a = -0.001
                if a < -0.999 :
                    a = -0.999
            else :
                if a < 0.001 :
                    a = 0.001
                if a > 0.999 :
                    a = 0.999
            LL_reg[j] = a
        if f_tot < 1.0 :
            y1.append(1 - y[i])
        else : 
            y1.append(y[i])
    return y1, LL_reg

In [249]:
def converged(m, cs_index, cs_list, C, y) :
    success = 1
    for i in range(m) :
        chksum = 0
        j1, j2 = cs_index[i], cs_index[i + 1]
        for j in range(j1, j2) :
            chksum = chksum ^ y[cs_list[j][0]]
        if chksum != C[i] :
            success = 0
    return success

In [250]:
def belief_prop1(C, D, MAX_ITERS, Na, Nf, Ma, Mf, p, k, y, H) :
    a2f, f2a = produce_a2f(p, Na, Nf, Ma, Mf), produce_f2a(p, Na, Nf, Ma, Mf)
    f_init, LL_reg = LL_init1(Na, Nf, k, p)
    LL_index, cs_index, cs_list = lookup_tables(H)
    success = 0
    i = 0
    while (i < MAX_ITERS) :
        LL_reg = cs_msgs_2_bits1(m, d, cs_index, cs_list, LL_reg, Ma)
        y, LL_reg = bit_msgs_2_cs1(n, f_init, a2f, Mf, LL_reg, y)
        success = converged(m, cs_index, cs_list, C)
        i += 1
        if success == 1 : 
            break
    return y, success

In [251]:
def belief_prop(C, D, y, MAX_ITERS, p, H) :
    m, n, k = len(H), len(H[0]), non_zeros(H)
    f_init, LL_reg = LL_init(k, p)
    LL_index, cs_index, cs_list = lookup_tables(H, m, n, k)
    d = [C[i] ^ D[i] for i in range(len(C))]
    
    success = 0
    i = 0
    while (i < MAX_ITERS) :
        LL_reg = cs_msgs_2_bits(m, d, cs_index, cs_list, LL_reg)
        y, LL_reg = bit_msgs_2_cs(n, LL_reg, LL_index, f_init, y)
        success = converged(m, cs_index, cs_list, C, y)
        i += 1
        if success == 1 : 
            break
    return y, success, i

In [252]:
ram_bits = [1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0]
sita_bits = [0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0]
n = 30
MAX_ITERS = 100
qber = 2/30
p = qber

In [253]:
#Step1 : Produce parity check matrix(M1) of dimension m*n, n is length of bitstring and m is no. of parity check equations
H, m = parity_matrix(n, p)

#Step2 : Ram produces the syndrome and hash values
C = syndrome(H, ram_bits)

#Step3 : Ram sends syndrome via CAC

#Step4 : Sita produces the syndrome
D = syndrome(H, sita_bits)


#Step5 : Ram performs belief propagation algorithm
belief_prop(C, D, sita_bits, MAX_ITERS, p, H)

#Step6 : Sita sends success of reconciliation


([1,
  1,
  1,
  0,
  1,
  0,
  1,
  0,
  1,
  0,
  1,
  0,
  1,
  0,
  1,
  0,
  1,
  0,
  1,
  0,
  0,
  0,
  0,
  1,
  1,
  0,
  0,
  1,
  0,
  0],
 1,
 1)